In [1]:
import chatbot

In [2]:
index, metadata = chatbot.load_single_index(chatbot.vector_dir)
print("Index and metadata loaded.")

Index and metadata loaded.


In [8]:
eval_dataset = [
  {
    "question": "O que é o Programa Temático Demografia, Qualificações e Inclusão no âmbito do PT2030?",
    "answer": "O Programa Temático Demografia, Qualificações e Inclusão do PT2030 tem como objetivo promover a inclusão social, melhorar as qualificações da população e responder aos desafios demográficos em Portugal. O programa apoia iniciativas nas áreas do emprego, educação, formação profissional, inclusão social e combate às desigualdades."
  },
  {
    "question": "Quais são os principais objetivos do Programa Temático Inovação e Transição Digital?",
    "answer": "O Programa Temático Inovação e Transição Digital visa reforçar a competitividade da economia portuguesa através do apoio à inovação, investigação e desenvolvimento, digitalização das empresas, adoção de tecnologias avançadas e capacitação digital dos trabalhadores."
  },
  {
    "question": "Que áreas são apoiadas pelo Programa Temático para a Ação Climática e Sustentabilidade?",
    "answer": "Este programa apoia investimentos relacionados com a transição climática, eficiência energética, energias renováveis, adaptação às alterações climáticas, proteção do ambiente e uso sustentável dos recursos naturais, contribuindo para uma economia mais verde e resiliente."
  },
  {
    "question": "O que é o MAR2030 no contexto do PT2030?",
    "answer": "O MAR2030 é o programa que operacionaliza o Fundo Europeu dos Assuntos Marítimos, das Pescas e Aquicultura em Portugal. Tem como objetivo promover a sustentabilidade ambiental, económica e social dos setores das pescas e da aquicultura, bem como o desenvolvimento das comunidades costeiras."
  },
  {
    "question": "Qual é o objetivo do Programa Regional do Norte 2021-2027?",
    "answer": "O Programa Regional do Norte 2021-2027 visa promover o desenvolvimento económico, social e territorial da região Norte, apoiando a inovação, a competitividade das empresas, a transição climática, a inclusão social e a coesão territorial."
  },
  {
    "question": "Que prioridades orientam o Programa Regional do Centro 2021-2027?",
    "answer": "O Programa Regional do Centro 2021-2027 apoia o crescimento sustentável da região Centro, com foco na inovação, valorização do território, transição energética, melhoria das qualificações e reforço da coesão social."
  },
  {
    "question": "Quais são os principais eixos do Programa Regional de Lisboa 2021-2027?",
    "answer": "O Programa Regional de Lisboa 2021-2027 centra-se na inovação, transição digital, sustentabilidade ambiental e inclusão social, procurando responder aos desafios específicos de uma região urbana e metropolitana."
  },
  {
    "question": "O que distingue o Programa Regional do Alentejo 2021-2027?",
    "answer": "O Programa Regional do Alentejo 2021-2027 aposta na valorização dos recursos endógenos, na sustentabilidade ambiental, no apoio às PME, na inovação e no reforço da coesão territorial numa região de baixa densidade populacional."
  },
  {
    "question": "Quais são os objetivos do Programa Regional do Algarve 2021-2027?",
    "answer": "O Programa Regional do Algarve 2021-2027 visa diversificar a economia regional, reduzir a sazonalidade, promover a sustentabilidade ambiental, apoiar a inovação e reforçar a resiliência do território."
  },
  {
    "question": "Qual é o papel do Programa Regional dos Açores 2021-2027?",
    "answer": "O Programa Regional dos Açores 2021-2027 apoia o desenvolvimento sustentável da região autónoma, promovendo a inovação, a adaptação às alterações climáticas, a mobilidade, a inclusão social e a valorização dos recursos naturais."
  },
  {
    "question": "Que objetivos tem o Programa Regional da Madeira 2021-2027?",
    "answer": "O Programa Regional da Madeira 2021-2027 tem como objetivo reforçar a competitividade regional, apoiar a transição digital e climática, melhorar a qualidade de vida da população e promover a coesão social e territorial."
  },
  {
    "question": "Para que serve o Programa de Assistência Técnica no PT2030?",
    "answer": "O Programa de Assistência Técnica apoia a gestão, monitorização, avaliação, comunicação e controlo dos programas do PT2030, garantindo a sua implementação eficaz e transparente."
  }
]

In [9]:
results = []

for item in eval_dataset:
    query = item["question"]
    gold_answer = item["answer"]
    
    final_answer, context_chunks, distances = chatbot.answer(query, index, metadata, k=5)
    
    results.append({
        "query": query,
        "gold": gold_answer,
        "llm_answer": final_answer,
        "retrieved_chunks": context_chunks,
        "distances": distances
    })


O Programa Temático Demografia, Qualificações e Inclusão no âmbito do PT2030 visa promover uma sociedade mais inclusiva e coesa, abordando questões relacionadas com a demografia, a qualificação da população e a inclusão social. Este programa tem como objetivos principais:

1. **Promoção da Inclusão Ativa**: Favorecer a igualdade de oportunidades e a não discriminação, especialmente para grupos desfavorecidos, através de ações que melhorem a empregabilidade e a participação ativa na sociedade.

2. **Qualificação da População**: Apostar na formação de jovens e adultos com baixos níveis de qualificação, visando a elevação das suas competências e a inserção no mercado de trabalho.

3. **Apoio a Grupos Vulneráveis**: Implementar medidas específicas para a inclusão de grupos em risco, como jovens em situação de abandono escolar, pessoas com deficiência, e populações imigrantes e ciganas.

4. **Transição Verde e Digital**: Integrar a formação e capacitação em áreas que respondam aos desafios 

In [ ]:
import numpy as np

def to_1d(vec):
    vec = np.array(vec)
    return vec.reshape(-1)

def cosine_sim(a, b):
    a = to_1d(a)
    b = to_1d(b)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))



def precision_recall_at_k(gold_answer, retrieved_chunks, threshold=0.75):
    gold_emb = chatbot.embed_query(gold_answer)

    relevant = 0
    for chunk in retrieved_chunks:
        chunk_emb = chatbot.embed_query(chunk["text"])
        if cosine_sim(gold_emb, chunk_emb) >= threshold:
            relevant += 1

    precision = relevant / len(retrieved_chunks) if retrieved_chunks else 0
    recall = 1.0 if relevant > 0 else 0.0

    return precision, recall



def answer_similarity(gold_answer, llm_answer):
    gold_emb = chatbot.embed_query(gold_answer)
    llm_emb = chatbot.embed_query(llm_answer)

    return cosine_sim(gold_emb, llm_emb)

def groundedness_score(llm_answer, retrieved_chunks):
    llm_emb = chatbot.embed_query(llm_answer)

    scores = []
    for chunk in retrieved_chunks:
        chunk_emb = chatbot.embed_query(chunk["text"])
        scores.append(cosine_sim(llm_emb, chunk_emb))

    return max(scores) if scores else 0



In [20]:
emb = chatbot.embed_query("teste")
print(np.array(emb).shape)


(1, 1536)


In [21]:
evaluated_results = []

for item in results:
    precision, recall = precision_recall_at_k(
        item["gold"],
        item["retrieved_chunks"]
    )

    ans_sim = answer_similarity(
        item["gold"],
        item["llm_answer"]
    )

    groundedness = groundedness_score(
        item["llm_answer"],
        item["retrieved_chunks"]
    )

    evaluated_results.append({
        "query": item["query"],
        "precision@k": precision,
        "recall@k": recall,
        "answer_similarity": ans_sim,
        "groundedness": groundedness
    })


In [26]:
import pandas as pd

df = pd.DataFrame(evaluated_results)

df.describe()


,precision@k,recall@k,answer_similarity,groundedness
count,12.000000,12.000000,12.000000,12.000000
mean,0.133333,0.416667,0.826912,0.793809
std,0.196946,0.514929,0.058709,0.047462
min,0.000000,0.000000,0.680908,0.720851
25%,0.000000,0.000000,0.823209,0.762601
50%,0.000000,0.000000,0.840135,0.783323
75%,0.200000,1.000000,0.862544,0.824184
max,0.600000,1.000000,0.894452,0.879966


You can explicitly say:

Increasing k:

↑ Recall

↓ Precision

↑ Groundedness (usually)

Better chunking:

↑ Precision

↑ Groundedness

Larger chunks:

↑ Recall

↓ Precision

These trade-offs show deep understanding.